In [ ]:
# Execute the complete pipeline
# This will test all hyperparameter combinations and find the best model

results, summary = main()

## Execute the Pipeline

**Run the cell below to start the complete hyperparameter search and evaluation:**

In [ ]:
# Main execution workflow
def main():
    """Main function to run the complete hyperparameter search and evaluation"""
    print(f"🚀 Starting DINOv3 Lake Detection - Production Pipeline")
    print(f"=" * 60)
    
    # Save configuration
    config_path = os.path.join(config.output_dir, "config.json")
    config.save_config(config_path)
    
    # Run complete hyperparameter search
    all_results = run_full_hyperparameter_search(image_rgb, mask_binary, boundary_mask, config)
    
    if not all_results:
        print("❌ No successful experiments completed")
        return
    
    # Create results summary
    summary_df = create_results_summary(all_results)
    
    # Save summary
    summary_path = os.path.join(config.output_dir, "experiments_summary.csv")
    summary_df.to_csv(summary_path, index=False)
    print(f"\\n✅ Experiments summary saved: {summary_path}")
    
    # Display results
    print(f"\\n📊 EXPERIMENTS SUMMARY:")
    print(summary_df.to_string(index=False, float_format='%.4f'))
    
    # Find best model
    best_experiment = summary_df.iloc[0]
    print(f"\\n🏆 BEST MODEL:")
    print(f"   Experiment: {best_experiment['Experiment']}")
    print(f"   Patch Size: {best_experiment['Patch Size']}")
    print(f"   Stride: {best_experiment['Stride']}")
    print(f"   Threshold: {best_experiment['Best Threshold']}")
    print(f"   IoU: {best_experiment['IoU']:.4f}")
    print(f"   Lake Count Error: {best_experiment['Count Error']}")
    
    # Create visualizations
    plot_hyperparameter_results(
        all_results, 
        save_path=os.path.join(config.output_dir, "hyperparameter_comparison.png")
    )
    
    # Plot training history for best model
    best_result = all_results[0]  # Assuming sorted by performance
    plot_training_history(
        best_result['training_history'],
        save_path=os.path.join(config.output_dir, "best_model_training_history.png")
    )
    
    print(f"\\n🎉 Production pipeline completed successfully!")
    print(f"   All results saved to: {config.output_dir}")
    
    return all_results, summary_df

print(f"✅ Main execution workflow defined")
print(f"\\n📋 Ready to run! Execute the main() function to start the pipeline.")

## Step 13: Main Execution Workflow

In [ ]:
# Results visualization and analysis
def plot_training_history(history, save_path=None):
    """Plot training and validation curves"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss curves
    axes[0, 0].plot(history['train_loss'], label='Train Loss', color='blue')
    axes[0, 0].plot(history['val_loss'], label='Validation Loss', color='red')
    axes[0, 0].set_title('Training and Validation Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # IoU curve
    axes[0, 1].plot(history['val_iou'], label='Validation IoU', color='green')
    axes[0, 1].set_title('Validation IoU')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('IoU')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Accuracy curve
    axes[1, 0].plot(history['val_accuracy'], label='Validation Accuracy', color='orange')
    axes[1, 0].set_title('Validation Accuracy')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # Combined metrics
    axes[1, 1].plot(history['val_iou'], label='IoU', color='green')
    axes[1, 1].plot(history['val_accuracy'], label='Accuracy', color='orange')
    axes[1, 1].set_title('Validation Metrics')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Score')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Training curves saved: {save_path}")
    
    plt.show()


def plot_hyperparameter_results(all_results, save_path=None):
    """Plot comparison of hyperparameter experiments"""
    if not all_results:
        print("No results to plot")
        return
    
    # Extract data for plotting
    experiments = []
    ious = []
    accuracies = []
    lake_counts_error = []
    
    for result in all_results:
        config = result['config']
        metrics = result['final_metrics']
        
        experiment_label = f"P{config['patch_size']}_S{config['stride']}_T{config['best_threshold']}"
        experiments.append(experiment_label)
        ious.append(metrics['pixel_iou'])
        accuracies.append(metrics['pixel_accuracy'])
        lake_counts_error.append(metrics['object_metrics']['count_error'])
    
    # Create comparison plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # IoU comparison
    axes[0, 0].bar(experiments, ious, color='skyblue')
    axes[0, 0].set_title('IoU Comparison Across Experiments')
    axes[0, 0].set_ylabel('IoU')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Accuracy comparison
    axes[0, 1].bar(experiments, accuracies, color='lightgreen')
    axes[0, 1].set_title('Accuracy Comparison Across Experiments')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Lake count error
    axes[1, 0].bar(experiments, lake_counts_error, color='salmon')
    axes[1, 0].set_title('Lake Count Error Across Experiments')
    axes[1, 0].set_ylabel('Count Error')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Combined scatter plot
    axes[1, 1].scatter(ious, accuracies, s=100, c=lake_counts_error, cmap='viridis')
    for i, exp in enumerate(experiments):
        axes[1, 1].annotate(exp, (ious[i], accuracies[i]), xytext=(5, 5), 
                           textcoords='offset points', fontsize=8)
    axes[1, 1].set_xlabel('IoU')
    axes[1, 1].set_ylabel('Accuracy')
    axes[1, 1].set_title('IoU vs Accuracy (Color = Count Error)')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Hyperparameter comparison saved: {save_path}")
    
    plt.show()


def create_results_summary(all_results):
    """Create summary table of all experiments"""
    summary_data = []
    
    for result in all_results:
        config = result['config']
        metrics = result['final_metrics']
        
        summary_data.append({
            'Experiment': result['experiment_name'],
            'Patch Size': config['patch_size'],
            'Stride': config['stride'],
            'Best Threshold': config['best_threshold'],
            'IoU': metrics['pixel_iou'],
            'Accuracy': metrics['pixel_accuracy'],
            'True Lakes': metrics['object_metrics']['true_lake_count'],
            'Pred Lakes': metrics['object_metrics']['pred_lake_count'],
            'Count Error': metrics['object_metrics']['count_error'],
            'Loading Test': result['loading_test_passed']
        })
    
    df = pd.DataFrame(summary_data)
    
    # Sort by IoU (best first)
    df = df.sort_values('IoU', ascending=False)
    
    return df

print(f"✅ Results analysis and visualization system defined")

## Step 12: Results Analysis and Visualization

In [ ]:
# Full image evaluation with boundary constraints
def predict_full_image(model, image, boundary_mask, patch_size, stride):
    """Apply model to full image using sliding window with boundary constraints"""
    model.eval()
    height, width = image.shape[:2]
    
    # Initialize output arrays
    full_mask = np.zeros((height, width), dtype=np.float32)
    count_mask = np.zeros((height, width), dtype=np.float32)
    
    # Image preprocessing
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    patches_processed = 0
    patches_skipped = 0
    
    with torch.no_grad():
        pbar = tqdm(
            range(0, height - patch_size + 1, stride),
            desc=f"Processing patches"
        )
        
        for y in pbar:
            for x in range(0, width - patch_size + 1, stride):
                # Check boundary constraint
                center_y = y + patch_size // 2
                center_x = x + patch_size // 2
                
                if boundary_mask[center_y, center_x] == 0:
                    patches_skipped += 1
                    continue
                
                # Extract and process patch
                patch = image[y:y+patch_size, x:x+patch_size, :3]
                
                # Resize to 224x224 for DINOv3
                patch_224 = cv2.resize(patch, (224, 224))
                
                # Transform and predict
                patch_tensor = transform(patch_224).unsqueeze(0).to(device)
                pred_224 = model(patch_tensor).squeeze().cpu().numpy()
                
                # Resize back to original patch size
                pred_patch = cv2.resize(pred_224, (patch_size, patch_size))
                
                # Accumulate predictions
                full_mask[y:y+patch_size, x:x+patch_size] += pred_patch
                count_mask[y:y+patch_size, x:x+patch_size] += 1
                
                patches_processed += 1
                
                if patches_processed % 100 == 0:
                    pbar.set_postfix({
                        'processed': patches_processed,
                        'skipped': patches_skipped
                    })
    
    # Average overlapping predictions
    final_mask = np.divide(
        full_mask, count_mask, 
        out=np.zeros_like(full_mask), 
        where=count_mask!=0
    )
    
    # Apply boundary constraint to final result
    final_mask = final_mask * boundary_mask
    
    print(f"   Processed {patches_processed} patches, skipped {patches_skipped}")
    
    return final_mask


def evaluate_full_image(model, image, mask, boundary_mask, config):
    """Evaluate model on full image with smart patch processing"""
    print(f"🖼️ Evaluating on full image...")
    
    # Generate full image prediction
    predicted_mask = predict_full_image(
        model, image, boundary_mask, 
        config.current_patch_size, config.current_stride
    )
    
    # Pixel-level metrics
    pixel_iou, pixel_accuracy = calculate_metrics(
        torch.from_numpy(predicted_mask).unsqueeze(0).unsqueeze(0),
        torch.from_numpy(mask).unsqueeze(0).unsqueeze(0),
        threshold=config.current_threshold
    )
    
    # Object-based metrics
    object_metrics = calculate_object_metrics(
        predicted_mask, mask, threshold=config.current_threshold
    )
    
    # Combined metrics
    full_metrics = {
        'pixel_iou': pixel_iou,
        'pixel_accuracy': pixel_accuracy,
        'object_metrics': object_metrics
    }
    
    # Print results
    print(f"\\n📊 Full Image Evaluation Results:")
    print(f"   Pixel IoU: {pixel_iou:.4f}")
    print(f"   Pixel Accuracy: {pixel_accuracy:.4f}")
    print_object_metrics(object_metrics)
    
    return full_metrics

print(f"✅ Full image evaluation system defined")

## Step 11: Full Image Evaluation

In [ ]:
# Hyperparameter testing framework
def test_probability_thresholds(model, val_loader, thresholds):
    """Test different probability thresholds on validation set"""
    print(f"🎯 Testing probability thresholds: {thresholds}")
    
    # Get all predictions
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc="Getting predictions"):
            images = images.to(device)
            preds = model(images)
            all_preds.append(preds.cpu())
            all_targets.append(masks)
    
    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    
    # Test each threshold
    threshold_results = {}
    for threshold in thresholds:
        iou, accuracy = calculate_metrics(all_preds, all_targets, threshold)
        threshold_results[threshold] = {'iou': iou, 'accuracy': accuracy}
        print(f"   Threshold {threshold}: IoU={iou:.4f}, Accuracy={accuracy:.4f}")
    
    return threshold_results


def run_hyperparameter_experiment(image, mask, boundary_mask, config):
    """Run single experiment with current hyperparameters"""
    experiment_name = config.get_experiment_name()
    print(f"\\n🔬 Running experiment: {experiment_name}")
    
    # Create datasets with current patch size
    train_dataset, val_dataset = create_datasets(image, mask, boundary_mask, config)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    
    # Create and train model
    model = DINOv3LakeDetector(
        dinov3_model_name=config.dinov3_model,
        feature_dim=config.dinov3_feature_dim
    ).to(device)
    
    # Train model
    trained_model, history = train_model_with_validation(model, train_loader, val_loader, config)
    
    # Test different probability thresholds
    threshold_results = test_probability_thresholds(
        trained_model, val_loader, config.probability_thresholds
    )
    
    # Find best threshold
    best_threshold = max(threshold_results.items(), key=lambda x: x[1]['iou'])[0]
    config.current_threshold = best_threshold
    
    # Final evaluation with best threshold
    final_metrics = evaluate_full_image(
        trained_model, image, mask, boundary_mask, config
    )
    
    # Combine all results
    experiment_results = {
        'experiment_name': experiment_name,
        'config': {
            'patch_size': config.current_patch_size,
            'stride': config.current_stride,
            'best_threshold': best_threshold
        },
        'training_history': history,
        'threshold_results': threshold_results,
        'final_metrics': final_metrics
    }
    
    # Save model
    model_path = save_model(trained_model, config, history, final_metrics, experiment_name)
    
    # Test model loading
    test_input = next(iter(val_loader))[0][:1].to(device)  # Single sample for testing
    loading_success = test_model_loading(trained_model, model_path, test_input)
    experiment_results['loading_test_passed'] = loading_success
    
    return experiment_results


def run_full_hyperparameter_search(image, mask, boundary_mask, config):
    """Run complete hyperparameter search across all combinations"""
    print(f"🔍 Starting full hyperparameter search...")
    
    all_results = []
    
    # Test all combinations of patch sizes and strides
    for patch_size, stride in zip(config.patch_sizes, config.strides):
        config.update_experiment(patch_size, stride)
        
        try:
            results = run_hyperparameter_experiment(image, mask, boundary_mask, config)
            all_results.append(results)
        except Exception as e:
            print(f"❌ Experiment {config.get_experiment_name()} failed: {e}")
            continue
    
    # Save all results
    results_path = os.path.join(config.output_dir, "hyperparameter_search_results.json")
    with open(results_path, 'w') as f:
        json.dump(all_results, f, indent=2, default=str)
    
    print(f"✅ Hyperparameter search complete. Results saved to: {results_path}")
    
    return all_results

print(f"✅ Hyperparameter testing framework defined")

## Step 10: Hyperparameter Testing Framework

In [ ]:
# Robust model saving and loading system
def save_model(model, config, history, metrics, experiment_name):
    """Save model with all necessary information for inference"""
    save_path = os.path.join(config.output_dir, f"{experiment_name}.pth")
    
    # Create comprehensive save package
    save_package = {
        # Model weights (state_dict approach for compatibility)
        'model_state_dict': model.state_dict(),
        
        # Model configuration
        'model_config': {
            'dinov3_model': config.dinov3_model,
            'feature_dim': config.dinov3_feature_dim,
            'patch_size': config.current_patch_size,
            'stride': config.current_stride,
            'threshold': config.current_threshold
        },
        
        # Training information
        'training_info': {
            'num_epochs': config.num_epochs,
            'learning_rate': config.learning_rate,
            'batch_size': config.batch_size,
            'timestamp': datetime.now().isoformat()
        },
        
        # Performance metrics
        'metrics': metrics,
        
        # Training history
        'history': history,
        
        # Data information
        'data_info': {
            'image_path': config.image_path,
            'mask_path': config.mask_path,
            'shapefile_path': config.shapefile_path
        }
    }
    
    # Save the package
    torch.save(save_package, save_path)
    print(f"✅ Model saved: {save_path}")
    
    return save_path


def load_model(model_path, device='cpu'):
    """Load model and return model instance with configuration"""
    print(f"📂 Loading model from: {model_path}")
    
    # Load the save package
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    
    # Recreate model architecture
    model_config = checkpoint['model_config']
    model = DINOv3LakeDetector(
        dinov3_model_name=model_config['dinov3_model'],
        feature_dim=model_config['feature_dim']
    )
    
    # Load trained weights
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    
    print(f"✅ Model loaded successfully")
    print(f"   Patch size: {model_config['patch_size']}")
    print(f"   Stride: {model_config['stride']}")
    print(f"   Threshold: {model_config['threshold']}")
    
    return model, checkpoint


def test_model_loading(original_model, save_path, test_input):
    """Test that saved model loads correctly and produces same predictions"""
    print(f"🧪 Testing model loading...")
    
    # Get prediction from original model
    original_model.eval()
    with torch.no_grad():
        original_pred = original_model(test_input)
    
    # Load model and get prediction
    loaded_model, _ = load_model(save_path, device=test_input.device)
    with torch.no_grad():
        loaded_pred = loaded_model(test_input)
    
    # Compare predictions
    diff = torch.abs(original_pred - loaded_pred).max().item()
    
    if diff < 1e-6:
        print(f"✅ Model loading test PASSED (max diff: {diff:.2e})")
        return True
    else:
        print(f"❌ Model loading test FAILED (max diff: {diff:.2e})")
        return False

print(f"✅ Model saving/loading system defined")

## Step 9: Model Saving and Loading System

In [ ]:
# Enhanced object-based evaluation with scikit-image
def calculate_object_metrics(pred_mask, true_mask, threshold=0.5):
    """Calculate object-based metrics: lake count, size distribution, etc."""
    from skimage.measure import label, regionprops
    
    # Convert to binary masks
    pred_binary = (pred_mask > threshold).astype(np.uint8)
    true_binary = (true_mask > 0).astype(np.uint8)
    
    # Find connected components (individual lakes)
    pred_labeled = label(pred_binary)
    true_labeled = label(true_binary)
    
    # Get properties of each lake
    pred_props = regionprops(pred_labeled)
    true_props = regionprops(true_labeled)
    
    # Extract lake sizes (areas)
    pred_sizes = [prop.area for prop in pred_props]
    true_sizes = [prop.area for prop in true_props]
    
    # Calculate metrics
    metrics = {
        'true_lake_count': len(true_sizes),
        'pred_lake_count': len(pred_sizes),
        'true_total_area': sum(true_sizes),
        'pred_total_area': sum(pred_sizes),
        'true_mean_size': np.mean(true_sizes) if true_sizes else 0,
        'pred_mean_size': np.mean(pred_sizes) if pred_sizes else 0,
        'true_max_size': max(true_sizes) if true_sizes else 0,
        'pred_max_size': max(pred_sizes) if pred_sizes else 0,
        'count_error': abs(len(pred_sizes) - len(true_sizes)),
        'area_error': abs(sum(pred_sizes) - sum(true_sizes))
    }
    
    return metrics


def print_object_metrics(metrics):
    """Print object-based metrics in a readable format"""
    print(f"\n📊 Object-Based Evaluation:")
    print(f"   Lake Count - True: {metrics['true_lake_count']}, Predicted: {metrics['pred_lake_count']} (error: {metrics['count_error']})")
    print(f"   Total Area - True: {metrics['true_total_area']}, Predicted: {metrics['pred_total_area']} (error: {metrics['area_error']})")
    print(f"   Mean Size - True: {metrics['true_mean_size']:.1f}, Predicted: {metrics['pred_mean_size']:.1f}")
    print(f"   Max Size - True: {metrics['true_max_size']}, Predicted: {metrics['pred_max_size']}")

print(f"✅ Enhanced object-based evaluation system defined")

## Step 8: Object-Based Evaluation Metrics

In [ ]:
# Object-based evaluation (lake counting and size analysis)
def calculate_object_metrics(pred_mask, true_mask, threshold=0.5):
    """Calculate object-based metrics: lake count, size distribution, etc."""
    from skimage.measure import label, regionprops
    
    # Convert to binary masks
    pred_binary = (pred_mask > threshold).astype(np.uint8)
    true_binary = (true_mask > 0).astype(np.uint8)
    
    # Find connected components (individual lakes)
    pred_labeled = label(pred_binary)
    true_labeled = label(true_binary)
    
    # Get properties of each lake
    pred_props = regionprops(pred_labeled)
    true_props = regionprops(true_labeled)
    
    # Extract lake sizes (areas)
    pred_sizes = [prop.area for prop in pred_props]
    true_sizes = [prop.area for prop in true_props]
    
    # Calculate metrics
    metrics = {
        'true_lake_count': len(true_sizes),
        'pred_lake_count': len(pred_sizes),
        'true_total_area': sum(true_sizes),
        'pred_total_area': sum(pred_sizes),
        'true_mean_size': np.mean(true_sizes) if true_sizes else 0,
        'pred_mean_size': np.mean(pred_sizes) if pred_sizes else 0,
        'true_max_size': max(true_sizes) if true_sizes else 0,
        'pred_max_size': max(pred_sizes) if pred_sizes else 0,
        'count_error': abs(len(pred_sizes) - len(true_sizes)),
        'area_error': abs(sum(pred_sizes) - sum(true_sizes))
    }
    
    return metrics


def print_object_metrics(metrics):
    """Print object-based metrics in a readable format"""
    print(f"\n📊 Object-Based Evaluation:")
    print(f"   Lake Count - True: {metrics['true_lake_count']}, Predicted: {metrics['pred_lake_count']} (error: {metrics['count_error']})")
    print(f"   Total Area - True: {metrics['true_total_area']}, Predicted: {metrics['pred_total_area']} (error: {metrics['area_error']})")
    print(f"   Mean Size - True: {metrics['true_mean_size']:.1f}, Predicted: {metrics['pred_mean_size']:.1f}")
    print(f"   Max Size - True: {metrics['true_max_size']}, Predicted: {metrics['pred_max_size']}")

print(f"✅ Object-based evaluation system defined")

# DINOv3 + U-Net Lake Detection - Production Version

## Overview
This notebook trains a deep learning model for pixel-level glacial lake detection using:
- **DINOv3**: Satellite-trained vision transformer as feature extractor (frozen)
- **U-Net Decoder**: Trainable segmentation head for precise lake boundaries
- **Smart Patch Strategy**: Small conceptual patches (16-32px) resized to 224px for DINOv3
- **Boundary Constraints**: Training only within glacier areas from shapefiles

## Key Features
- Proper train/validation split with performance tracking
- Hyperparameter testing (patch sizes, strides, thresholds)
- Object-based evaluation metrics (lake counting)
- Robust model saving/loading system
- Production-ready code structure

## Step 1: Environment Setup and Imports

In [ ]:
# Package installation for Google Colab
!pip install torch torchvision transformers accelerate
!pip install opencv-python pillow scikit-image
!pip install numpy scipy scikit-learn
!pip install rasterio geopandas fiona shapely
!pip install matplotlib seaborn
!pip install tqdm

print("✅ All packages installed successfully")

In [ ]:
# Core imports - organized by functionality
import os
import json
from datetime import datetime

# Scientific computing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Computer vision and image processing
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

# Geospatial processing
import rasterio
import geopandas as gpd
from rasterio.features import geometry_mask

# Deep learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# HuggingFace and DINOv3
from transformers import AutoModel
from huggingface_hub import login

# Progress tracking
from tqdm import tqdm

# Setup device and random seeds for reproducibility
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
np.random.seed(42)

print(f"✅ Device: {device}")
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

## Step 2: Configuration System

In [ ]:
# Configuration class for all hyperparameters
class Config:
    """Configuration class for DINOv3 lake detection training"""
    
    def __init__(self):
        # File paths
        self.image_path = '/content/drive/MyDrive/superlakes/2021-09-04_fcc_testclip2.tif'
        self.mask_path = '/content/drive/MyDrive/superlakes/lake_mask_testclip2.tif'
        self.shapefile_path = '/content/drive/MyDrive/superlakes/vectors/clip_by_glacier.shp'
        self.output_dir = '/content/drive/MyDrive/superlakes_production/'
        
        # HuggingFace model
        self.hf_token = "insert-hf-token-here"
        self.dinov3_model = "facebook/dinov3-vitb16-pretrain-lvd1689m"
        
        # Model architecture
        self.dinov3_feature_dim = 768
        self.dinov3_input_size = 224
        
        # Training parameters
        self.batch_size = 4
        self.num_epochs = 10
        self.learning_rate = 0.001
        self.val_split = 0.2
        self.early_stopping_patience = 3
        
        # Dataset parameters
        self.max_patches = 5000
        self.lake_ratio = 0.7  # Ratio of lake patches to background patches
        self.min_lake_coverage = 0.01  # Minimum lake coverage in patch (1%)
        
        # Hyperparameters to test
        self.patch_sizes = [16, 20, 32]
        self.strides = [8, 10, 16]  # Corresponding to patch sizes
        self.probability_thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
        
        # Current experiment settings (will be updated during hyperparameter search)
        self.current_patch_size = 16
        self.current_stride = 8
        self.current_threshold = 0.5
    
    def update_experiment(self, patch_size, stride, threshold=0.5):
        """Update current experiment parameters"""
        self.current_patch_size = patch_size
        self.current_stride = stride
        self.current_threshold = threshold
    
    def get_experiment_name(self):
        """Generate experiment name for saving results"""
        return f"dinov3_patch{self.current_patch_size}_stride{self.current_stride}_thr{self.current_threshold}"
    
    def save_config(self, filepath):
        """Save configuration to JSON file"""
        config_dict = {k: v for k, v in self.__dict__.items() if not k.startswith('hf_token')}
        with open(filepath, 'w') as f:
            json.dump(config_dict, f, indent=2, default=str)
        print(f"✅ Configuration saved to {filepath}")

# Initialize configuration
config = Config()
print(f"✅ Configuration initialized")
print(f"   Current experiment: {config.get_experiment_name()}")

## Step 3: Google Drive Setup and Data Loading

In [ ]:
# Mount Google Drive and authenticate with HuggingFace
from google.colab import drive
drive.mount('/content/drive')

# Authenticate with HuggingFace for DINOv3 access
login(token=config.hf_token)

# Create output directory
os.makedirs(config.output_dir, exist_ok=True)
print(f"✅ Output directory: {config.output_dir}")

In [ ]:
# Load satellite image and manual lake mask
def load_data(image_path, mask_path):
    """Load satellite image and ground truth mask"""
    print(f"📂 Loading data...")
    
    # Load satellite image
    with rasterio.open(image_path) as src:
        image = src.read()  # Shape: (channels, height, width)
        image = np.transpose(image, (1, 2, 0))  # Convert to (height, width, channels)
        image_profile = src.profile.copy()
    
    # Load ground truth mask
    with rasterio.open(mask_path) as src:
        mask = src.read(1)  # Read first band
    
    # Convert to appropriate formats
    image_rgb = image[:,:,:3].astype(np.uint8)  # Use only RGB channels
    mask_binary = (mask > 0).astype(np.float32)  # Binary mask for training
    
    print(f"   Image shape: {image_rgb.shape}")
    print(f"   Mask shape: {mask_binary.shape}")
    print(f"   Image value range: {image_rgb.min()} - {image_rgb.max()}")
    print(f"   Lake pixels: {mask_binary.sum():,.0f} ({mask_binary.mean()*100:.2f}%)")
    
    return image_rgb, mask_binary, image_profile

# Load the data
image_rgb, mask_binary, image_profile = load_data(config.image_path, config.mask_path)

## Step 4: Boundary Mask Creation

In [ ]:
# Create boundary mask from glacier shapefile
def create_boundary_mask(image_path, shapefile_path):
    """Create binary mask from shapefile to constrain analysis to glacier areas"""
    print(f"🗺️ Creating boundary mask from shapefile...")
    
    # Load shapefile
    gdf = gpd.read_file(shapefile_path)
    print(f"   Loaded {len(gdf)} polygon(s) from shapefile")
    
    # Get image spatial information
    with rasterio.open(image_path) as src:
        transform = src.transform
        shape = src.shape
        crs = src.crs
    
    print(f"   Image CRS: {crs}")
    print(f"   Shapefile CRS: {gdf.crs}")
    
    # Reproject shapefile to match image CRS if needed
    if gdf.crs != crs:
        print(f"   Reprojecting shapefile from {gdf.crs} to {crs}")
        gdf = gdf.to_crs(crs)
    
    # Create binary mask (True inside polygons, False outside)
    boundary_mask = ~geometry_mask(
        gdf.geometry, 
        transform=transform, 
        invert=False, 
        out_shape=shape
    )
    
    pixels_inside = boundary_mask.sum()
    percentage_inside = pixels_inside / boundary_mask.size * 100
    
    print(f"   Boundary mask created: {pixels_inside:,} pixels inside ({percentage_inside:.1f}%)")
    
    return boundary_mask.astype(np.uint8)

# Create boundary mask
boundary_mask = create_boundary_mask(config.image_path, config.shapefile_path)

## Step 5: Model Architecture Definition

In [ ]:
# U-Net Decoder for pixel-level segmentation
class UNetDecoder(nn.Module):
    """U-Net decoder that converts DINOv3 features to segmentation masks"""
    
    def __init__(self, feature_dim=768, target_size=224):
        super(UNetDecoder, self).__init__()
        self.target_size = target_size
        
        # Progressive upsampling layers
        self.conv1 = nn.Conv2d(feature_dim, 512, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.final_conv = nn.Conv2d(64, 1, kernel_size=1)
        
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # Apply convolutional layers with ReLU activation
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.final_conv(x)
        
        # Ensure output is exactly the target size
        x = nn.functional.interpolate(
            x, size=(self.target_size, self.target_size), 
            mode='bilinear', align_corners=False
        )
        
        # Apply sigmoid for probability output
        x = self.sigmoid(x)
        return x


# Complete DINOv3 + U-Net model
class DINOv3LakeDetector(nn.Module):
    """Complete model: DINOv3 feature extractor + U-Net decoder for lake segmentation"""
    
    def __init__(self, dinov3_model_name, feature_dim=768):
        super(DINOv3LakeDetector, self).__init__()
        
        # Load pre-trained DINOv3 model (frozen for feature extraction)
        self.dinov3 = AutoModel.from_pretrained(dinov3_model_name)
        
        # Freeze DINOv3 parameters (use as feature extractor only)
        for param in self.dinov3.parameters():
            param.requires_grad = False
        
        # Trainable U-Net decoder
        self.decoder = UNetDecoder(feature_dim=feature_dim, target_size=224)
        
        print(f"✅ Model created: DINOv3 (frozen) + U-Net decoder (trainable)")
    
    def forward(self, x):
        # Extract features with DINOv3 (no gradients)
        with torch.no_grad():
            features = self.dinov3(x).last_hidden_state
            
            # Remove CLS token and reshape to spatial feature map
            patch_features = features[:, 1:]  # Remove first token (CLS)
            batch_size, num_patches, feature_dim = patch_features.shape
            
            # Calculate spatial dimensions (assume square patches)
            spatial_size = int(num_patches ** 0.5)
            
            # Reshape to 2D feature map
            feature_map = patch_features.reshape(
                batch_size, spatial_size, spatial_size, feature_dim
            )
            feature_map = feature_map.permute(0, 3, 1, 2)  # (B, C, H, W)
        
        # Generate segmentation mask with trainable decoder
        mask = self.decoder(feature_map)
        return mask

print(f"✅ Model architecture defined")

## Step 6: Dataset Creation with Train/Validation Split

In [ ]:
# Dataset class for boundary-constrained lake detection
class LakeDetectionDataset(Dataset):
    """Dataset that samples patches only inside glacier boundaries"""
    
    def __init__(self, image, mask, boundary_mask, patch_size, max_patches, lake_ratio):
        self.patch_size = patch_size
        self.dinov3_size = 224
        self.patches = []
        self.mask_patches = []
        
        # Data transforms for DINOv3 (ImageNet normalization)
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        self._create_patches(image, mask, boundary_mask, max_patches, lake_ratio)
    
    def _create_patches(self, image, mask, boundary_mask, max_patches, lake_ratio):
        """Extract patches from image constrained by boundary mask"""
        height, width = image.shape[:2]
        stride = self.patch_size // 8  # Dense sampling for better coverage
        
        print(f"   Creating {self.patch_size}x{self.patch_size} patches with stride {stride}...")
        
        lake_patches = []
        background_patches = []
        
        # Sample patches within boundary
        for y in range(0, height - self.patch_size + 1, stride):
            for x in range(0, width - self.patch_size + 1, stride):
                # Check if patch center is inside boundary
                center_y = y + self.patch_size // 2
                center_x = x + self.patch_size // 2
                
                if boundary_mask[center_y, center_x]:
                    # Calculate lake coverage in this patch
                    mask_patch = mask[y:y+self.patch_size, x:x+self.patch_size]
                    lake_coverage = mask_patch.mean()
                    
                    patch_info = {
                        'y': y, 'x': x, 'coverage': lake_coverage
                    }
                    
                    if lake_coverage > config.min_lake_coverage:
                        lake_patches.append(patch_info)
                    elif lake_coverage == 0:
                        background_patches.append(patch_info)
        
        print(f"   Found {len(lake_patches)} lake patches, {len(background_patches)} background patches")
        
        # Create balanced dataset
        n_lake = min(len(lake_patches), int(max_patches * lake_ratio))
        n_background = min(len(background_patches), max_patches - n_lake)
        
        print(f"   Selecting {n_lake} lake patches and {n_background} background patches")
        
        # Sort lake patches by coverage (highest first) and select top ones
        lake_patches.sort(key=lambda x: x['coverage'], reverse=True)
        selected_patches = lake_patches[:n_lake] + np.random.choice(
            background_patches, n_background, replace=False
        ).tolist()
        
        # Extract actual image and mask patches
        for patch_info in selected_patches:
            y, x = patch_info['y'], patch_info['x']
            
            img_patch = image[y:y+self.patch_size, x:x+self.patch_size, :3]
            mask_patch = mask[y:y+self.patch_size, x:x+self.patch_size]
            
            self.patches.append(img_patch)
            self.mask_patches.append(mask_patch)
        
        print(f"   Final dataset: {len(self.patches)} patches")
    
    def __len__(self):
        return len(self.patches)
    
    def __getitem__(self, idx):
        # Get patch and resize to DINOv3 input size
        image_patch = self.patches[idx]
        mask_patch = self.mask_patches[idx]
        
        # Resize to 224x224 for DINOv3
        image_224 = cv2.resize(image_patch, (self.dinov3_size, self.dinov3_size))
        mask_224 = cv2.resize(mask_patch.astype(np.float32), (self.dinov3_size, self.dinov3_size))
        
        # Apply transforms
        image_tensor = self.transform(image_224)
        mask_tensor = torch.from_numpy(mask_224).float().unsqueeze(0)
        
        return image_tensor, mask_tensor


# Create dataset and split into train/validation
def create_datasets(image, mask, boundary_mask, config):
    """Create train and validation datasets with proper split"""
    print(f"📊 Creating datasets...")
    
    # Create full dataset
    full_dataset = LakeDetectionDataset(
        image, mask, boundary_mask,
        patch_size=config.current_patch_size,
        max_patches=config.max_patches,
        lake_ratio=config.lake_ratio
    )
    
    # Split into train and validation
    dataset_size = len(full_dataset)
    val_size = int(dataset_size * config.val_split)
    train_size = dataset_size - val_size
    
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_dataset, [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"   Train set: {len(train_dataset)} patches")
    print(f"   Validation set: {len(val_dataset)} patches")
    
    return train_dataset, val_dataset

print(f"✅ Dataset creation functions defined")

## Step 7: Training System with Validation

In [ ]:
# Training function with proper validation
def train_model_with_validation(model, train_loader, val_loader, config):
    """Train model with validation tracking and early stopping"""
    print(f"🚀 Starting training with validation...")
    
    # Setup training components
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.decoder.parameters(), lr=config.learning_rate)
    
    # Training history tracking
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_iou': [],
        'val_accuracy': []
    }
    
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(config.num_epochs):
        print(f"\n--- Epoch {epoch+1}/{config.num_epochs} ---")
        
        # Training phase
        model.train()
        train_loss = 0.0
        
        train_pbar = tqdm(train_loader, desc=f"Training")
        for batch_idx, (images, masks) in enumerate(train_pbar):
            images, masks = images.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_pbar.set_postfix({'loss': loss.item():.4f})
        
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        all_val_preds = []
        all_val_targets = []
        
        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f"Validation")
            for images, masks in val_pbar:
                images, masks = images.to(device), masks.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()
                
                # Collect predictions for metrics
                all_val_preds.append(outputs.cpu())
                all_val_targets.append(masks.cpu())
                
                val_pbar.set_postfix({'loss': loss.item():.4f})
        
        avg_val_loss = val_loss / len(val_loader)
        
        # Calculate validation metrics
        val_preds = torch.cat(all_val_preds, dim=0)
        val_targets = torch.cat(all_val_targets, dim=0)
        
        val_iou, val_accuracy = calculate_metrics(val_preds, val_targets, threshold=0.5)
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['val_iou'].append(val_iou)
        history['val_accuracy'].append(val_accuracy)
        
        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Val Loss: {avg_val_loss:.4f}")
        print(f"Val IoU: {val_iou:.4f}")
        print(f"Val Accuracy: {val_accuracy:.4f}")
        
        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
            print(f"✅ New best validation loss: {best_val_loss:.4f}")
        else:
            patience_counter += 1
            print(f"⏳ Patience: {patience_counter}/{config.early_stopping_patience}")
            
            if patience_counter >= config.early_stopping_patience:
                print(f"🛑 Early stopping triggered")
                break
    
    # Load best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"✅ Loaded best model (val_loss: {best_val_loss:.4f})")
    
    return model, history


# Evaluation metrics calculation
def calculate_metrics(pred_masks, true_masks, threshold=0.5):
    """Calculate IoU and pixel accuracy metrics"""
    pred_binary = (pred_masks > threshold).float()
    
    # IoU calculation
    intersection = (pred_binary * true_masks).sum()
    union = pred_binary.sum() + true_masks.sum() - intersection
    iou = intersection / (union + 1e-8)
    
    # Pixel accuracy
    accuracy = (pred_binary == true_masks).float().mean()
    
    return iou.item(), accuracy.item()

print(f"✅ Training system defined")